In [49]:
import os
import sys
import logging
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from dotenv import load_dotenv

load_dotenv()

# API 키 설정
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    logging.error("API 키를 설정해주세요.")

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=1.0)

In [50]:
# === 문서 로드 및 인덱스 생성 (RAG) ===
# 문서 폴더 내에는 아래 데이터가 포함되어 있다고 가정.
# 1. 합격 자소서 데이터, 2. 합격자 면접 데이터, 3. 업계 동향 데이터, 4. 업계 구인 데이터

# 문서 로드를 위한 패키지
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Document
from llama_index.core import GPTVectorStoreIndex

# 문서 로드 및 인덱스 생성
data_path = os.getenv("DATA_PATH")
documents = SimpleDirectoryReader(data_path).load_data()
if not data_path:
    logging.error("데이터 경로를 설정해주세요.")

index = GPTVectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [51]:
# 참고 자료 검색: 자소서 분석 및 피드백에 유용한 자료를 외부 문서에서 추출
reference_query = (
    "합격 자소서, 합격자 면접, 업계 동향, 구인 데이터를 참고하여, "
    "자소서 작성 피드백에 유용한 핵심 인사이트를 간략히 요약해줘."
)
reference_result = query_engine.query(reference_query)

# 지원 기업 관련 산업 뉴스 데이터 추출 (지원 기업 및 관련 키워드 활용)
company = "A기업"
industry_news_query = (
    f"{company} 관련 산업 뉴스, 최신 동향, 전략 및 이슈"
)
industry_news_result = query_engine.query(industry_news_query)


# 두 결과를 결합하여 참고 자료로 활용
combined_reference = f"{reference_result}\n\n[산업 뉴스 참고]:\n{industry_news_result}"


In [80]:
## 여기부터 실행

# === 자소서 분석 체인 프롬프트 템플릿 ===
init_prompt = PromptTemplate(
    input_variables=["참고자료"],
    template="""
    이전 대화 내용을 무시하고, 아래에 제공된 정보만을 기반으로 분석을 수행해 주세요.
    이후에 입력하는 내용을 기반으로 자소서 분석, 조언, 피드백을 제공해주세요.
    --------------------------------------
    {참고자료} 
    --------------------------------------
    """
)

In [81]:
# === 수정된 분석 체인 프롬프트 템플릿 (회사, 직무 정보 포함 및 적합성 체크) ===
analysis_prompt = PromptTemplate(
    input_variables=["자소서내용", "참고자료", "회사", "직무"],
    template="""
사용자가 입력한 자소서 내용은 다음과 같습니다:
--------------------------------------
{자소서내용}
--------------------------------------
아래의 참고 자료는 합격 자소서, 면접 데이터, 업계 동향, 구인 데이터 및 지원 기업 관련 산업 뉴스에서 추출된 인사이트입니다:
--------------------------------------
{참고자료}
--------------------------------------
또한, 사용자가 지원하는 회사는 "{회사}"이며 희망 직무는 "{직무}"입니다.
위 정보를 종합하여, 다음 사항들을 분석하고, 예상 합격/불합격 여부를 알려주고 점수도 내줘(예 : 8.0/10 ).:
자소서에 나타난 강점과 약점을 각각 3가지씩 구체적으로 제시
자소서 내용이 지원하는 회사와 희망 직무에 얼마나 부합하는지 평가하고, 개선이 필요한 부분(예: 지원 동기, 관련 경험, 역량 표현 등)을 구체적으로 지적해줘.
예시) 강점: [문제 해결 능력, 팀워크 등] / 약점: [경험 부족, 구체성 부족 등] / 적합성 평가: [지원 동기 보완 필요 등]
"""
)

In [82]:
# === 조언 체인 프롬프트 템플릿 (회사, 직무 정보 추가) ===
advice_prompt = PromptTemplate(
    input_variables=["분석결과", "회사", "직무"],
    template="""
아래 분석 결과와 지원 정보(지원 회사 및 직무)를 참고하여, 자소서 개선 및 보완을 위한 구체적인 조언을 작성해줘.
분석 결과:
--------------------------------------
{분석결과}
--------------------------------------
지원 정보:
- 지원하는 회사: {회사}
- 지원하는 직무: {직무}

조언은 실제 자소서에 적용 가능한 수정 방안과 개선 방향을 상세하게 제시하고, 지원 회사와 직무에 맞는 어필 포인트를 반영해줘.
"""
)

In [83]:
# === 자기소개서 예시 출력 프롬프트 템플릿 (회사, 직무 정보 포함) ===
example_prompt = PromptTemplate(
    input_variables=["조언결과", "회사", "직무", "자기소개서"],
    template="""
아래 조언 결과를 바탕으로, 실제 지원자가 제출할 수 있는 자기소개서 예시를 작성해줘.
자기소개서는 다음과 같은 구조를 갖추어야 합니다:
1. **지원동기 및 비전:** 지원 동기, 본인의 비전, 그리고 지원 회사({회사}) 및 직무({직무})와의 시너지를 명확히 서술
2. **경험 및 역량:** 학습, 동아리/프로젝트 경험, 보유 기술 및 역량을 구체적으로 기술
3. **강점 및 보완 사항:** 분석된 강점을 부각시키고, 약점을 개선하기 위한 구체적인 실행 계획 제시
4. **향후 계획:** 입사 후 성장 방향과 목표

각 항목별로 소제목을 붙여 기존 자기소개서에 조언결과를 적용한 결과를 예시로 보여줘.
예시 자기소개서는 기존 자기소개서의 분량과 비슷하게 작성해줘.
아래 조언 결과:
--------------------------------------
{조언결과}
--------------------------------------
자기소개서 예시:
{자기소개서}
"""
)

In [84]:
# 체인 생성: 분석 체인, 조언 체인, 예시 출력 체인
init_chain = LLMChain(llm=llm, prompt=init_prompt)
analysis_chain = LLMChain(llm=llm, prompt=analysis_prompt)
advice_chain = LLMChain(llm=llm, prompt=advice_prompt)
example_chain = LLMChain(llm=llm, prompt=example_prompt)

In [85]:
# === 사용자의 입력 ===
user_self_intro = """
**지원 동기 및 비전: 최고의 사용자 경험을 만드는 소프트웨어 엔지니어로서의 꿈**  
현재 IT 산업은 빠르게 변화하고 있으며, 사용자들은 단순히 기능적 요구를 충족하는 것을 넘어 직관적이고 매끄러운 인터랙션을 기대하고 있습니다. 특히 가비아의 비전인 “advanced industries”에 깊이 공감하며, 고객이 다가갈 수 있는 모든 디지털 접점을 최적의 경험으로 이끄는 것에 기여하고 싶습니다. 전자공학을 전공하면서 하드웨어와 소프트웨어의 통합이 주는 가능성을 직간접적으로 경험하였고, 특히 사용자 중심의 개발이 얼마나 중요한지 깨닫게 되었습니다. 이를 바탕으로, 가비아의 소프트웨어 엔지니어로서 최신 기술을 활용한 최상의 사용자 경험을 제공하고 싶습니다.

**경험 및 역량: 다양한 프로젝트로 다져온 소프트웨어 개발 실력**  
저는 웹 개발에 집중하며 React, TypeScript 등을 활용한 여러 프로젝트를 진행했습니다. 이 중 <스마트 홈 관리 시스템> 프로젝트에서 사용자 피드백을 반영하여 UI/UX를 개선했으며, 이로 인해 사용자 만족도가 30% 향상되었습니다. 특히 Next.js를 활용하여 서버 사이드 렌더링(SSR)을 구현해 페이지 로딩 속도를 40% 줄이고 사용자 이탈률을 크게 낮춘 경험이 있습니다.

또한, 대학 시절 동아리에서 진행한 프로젝트를 통해 Atomic Design 패턴을 적용해 UI 컴포넌트를 설계했으며, Storybook을 활용하여 독립적인 컴포넌트 개발 및 테스트를 진행하는 방법을 익혔습니다. 이러한 경험은 재사용성과 유지보수성을 고려한 개발 능력을 키우는데 큰 도움이 되었습니다.

**강점 및 보완 사항: 협업 능력과 성과 중심의 접근**  
저의 가장 큰 강점은 사용자 피드백을 적극적으로 반영하여 개선하는 능력입니다. 특히 협업 과정에서 열린 마음으로 다른 개발자의 의견을 경청하고, 최적의 해결책을 도출하기 위해 노력합니다. 한편, 제 약점은 때로 지나친 완벽주의가 프로젝트 일정에 영향을 미칠 수 있다는 점입니다. 이를 보완하기 위해, 개인적으로 ‘고-로우’ 목표 설정 전략을 수립하고, 우선 실행할 기능과 개선할 사항을 차별화하여 보다 유연하게 프로젝트를 관리하고 있습니다.

**향후 계획: 가비아와 함께 성장하며 차별화된 사용자 경험 제공**  
가비아에서 제 목표는 프론트엔드 개발자로서 더욱 사용자 친화적이고 성능 최적화된 애플리케이션을 개발하는 것입니다. 특히, 고객의 요구를 실시간으로 반영할 수 있는 시스템 개선이나 새로운 기능 추가를 통해 사용자들이 더욱 편리하게 사용할 수 있도록 기여하고 싶습니다. 입사 후에는 팀원들과의 원활한 소통을 통해 협력적인 개발 환경을 만들어가며, 지속적인 자기 개발을 통해 최고의 소프트웨어 엔지니어로 성장하는 것이 제 비전입니다.

가비아에서의 기회를 통해 더 나은 사용자 경험을 창출하고, 함께 성장하는 모습을 보여드리겠습니다.
"""

# 추가 입력: 지원하는 회사와 직무
company = "가비아"
job_position = "소프트웨어 엔지니어"

In [86]:
# 0. 초기화 체인: 이전 프롬프트의 내용을 초기화하고, 새로운 참고 자료를 입력
init_result = init_chain.run(
    참고자료=combined_reference
)

In [87]:
# 1. 분석 체인 실행: 자소서 내용과 참고 자료, 지원 정보를 기반으로 강점/약점 및 적합성 평가
analysis_result = analysis_chain.run(
    자소서내용=user_self_intro,
    참고자료=combined_reference,
    회사=company,
    직무=job_position
)
print("=== 분석 결과 ===")
print(analysis_result)

=== 분석 결과 ===
### 강점
1. **사용자 중심의 접근**: 사용자가 기대하는 직관적 경험을 제공하고자 하는 강한 동기와 방향성을 가지고 있습니다. 이는 소프트웨어 엔지니어로서 필수적인 사용자 중심의 개발 철학과 일치합니다.
  
2. **프로젝트 경험과 성과**: 여러 프로젝트를 통해 실질적인 기술적 경험을 강조했습니다. 특히 UI/UX 개선 및 성과(30% 향상)와 서버 사이드 렌더링 구현을 통한 페이지 로딩 속도 개선(40%)과 같은 수치적 성과는 인상적입니다.

3. **협업 능력**: 열린 마음으로 다른 개발자의 의견을 경청하고, 문제 해결을 위해 서로 협력하려는 태도는 팀워크와 협업이 중요한 IT 산업에서 중요한 자산입니다.

### 약점
1. **완벽주의로 인한 일정 문제**: 지나친 완벽주의가 프로젝트 일정에 영향을 미칠 수 있다는 점은 소프트웨어 개발에 있어 팀원들과의 협력이 필요한 경우 문제가 될 수 있습니다.

2. **구체성 부족**: 일부 경험에 대한 설명이 상세하지 않으며, 예를 들어 프로젝트의 구체적인 역할 또는 기술 스택의 활용 사례가 부족합니다. 이는 소프트웨어 엔지니어의 기술적 역량을 명확히 보여주지 못합니다.

3. **전문성의 깊이 부족**: 특정 기술이나 패턴(예: Atomic Design 패턴)의 활용에 대한 설명이 일반적이며, 심화된 전문성이나 독창적인 접근방식을 제시하지 못했습니다.

### 적합성 평가
- **전반적인 적합성**: 자소서는 지원하는 회사인 가비아와 희망 직무인 소프트웨어 엔지니어에 대해 긍정적인 요소가 많습니다. 환영하는 기능적 기술 외에도 협업 능력과 사용자 경험을 중시하는 점은 회사의 비전과 잘 부합합니다.
  
- **지원 동기 보완 필요**: 회사의 비전에 대한 이해도와 개인적 비전을 보다 명확하게 연결지어 설명할 필요가 있습니다. 가비아가 집중하는 분야(예: 최신 디지털 기술)와 개인의 경로를 구체적으로 맞추어 보완하면 좋겠습니다.

- **경험과 역량 표현**: 경험에 대한 구체

In [88]:
# 2. 조언 체인 실행: 분석 결과와 지원 정보를 기반으로 개선 조언 생성
advice_result = advice_chain.run(
    분석결과=analysis_result,
    회사=company,
    직무=job_position
)
print("\n=== 조언 결과 ===")
print(advice_result)


=== 조언 결과 ===
자소서를 개선하고 보완하기 위해 다음과 같은 구체적인 조언을 드립니다. 가비아와 소프트웨어 엔지니어 직무에 맞춰 지원자로서의 강점을 더욱 부각시키고 약점을 해결해 보시길 바랍니다.

### 1. 지원 동기 및 회사에 대한 이해 강화

- **가비아의 비전과 미션 연결**: 자소서의 초반에 가비아의 비전(예: 최신 디지털 기술 및 사용자 경험 개선)에 대한 이해를 바탕으로, 지원자가 추구하는 목표와 연결시키는 문장을 작성하세요. 예를 들어, "가비아의 디지털 기술 혁신에 기여하고, 사용자가 기대하는 경험을 더욱 향상시키는 데 기여하고자 합니다."와 같은 방식으로 시작할 수 있습니다.

- **구체적인 프로젝트 사례**: 가비아에서 수행할 수 있는 특정 프로젝트(예: 클라우드 서비스 개선, UX/UI 혁신)와 관련된 본인의 경험을 엮어 강조하면 좋습니다. 예를 들어, "이전 프로젝트에서 사용자 피드백을 통해 UI를 30% 개선했듯이, 가비아의 고객 중심 접근 방식을 통해 더욱 향상된 사용자 경험을 제공할 수 있습니다."라는 식으로 구체적인 사례를 포함해 보세요.

### 2. 경험과 역량의 구체성 및 깊이 강화

- **프로젝트 설명의 디테일 추가**: 각 프로젝트에서의 구체적인 역할, 사용한 기술 스택, 문제 해결 과정 및 결과를 명확히 기술하세요. 예를 들어, "React와 Node.js를 사용하여 서버 사이드 렌더링을 구현하여 페이지 로딩 속도를 40% 향상시켰습니다. 이 과정에서 이슈를 빠르게 해결하기 위한 협업 방안을 제시했고, 팀원들과의 코드 리뷰를 통해 효율성을 높였습니다."라는 식으로 명확하게 기술합니다.

- **전문 기술 언급**: Atomic Design 패턴 등 전문 기술에 대한 개인적인 적용 사례를 제시하세요. "UI 컴포넌트를 Atomic Design 패턴에 따라 설계하여 코드 재사용성을 높이는 데 기여했습니다."와 같은 구체적인 설명을 더해 전문성을 보강합니다.

### 3. 협업 능력 강조

- **구체적인 

In [89]:
# 3. 예시 출력 체인 실행: 조언 결과와 지원 정보를 반영한 자기소개서 예시 생성
example_self_intro = example_chain.run(
    조언결과=advice_result,
    회사=company,
    직무=job_position,
    자기소개서=user_self_intro
)
print("\n=== 자기소개서 예시 ===")
print(example_self_intro)


=== 자기소개서 예시 ===
**지원 동기 및 비전: 최고의 사용자 경험을 만드는 소프트웨어 엔지니어로서의 꿈**  
IT 산업은 빠르게 변화하며 사용자들은 단순 기능을 넘어 직관적이고 매끄러운 인터랙션을 요구하고 있습니다. 가비아의 “advanced industries” 비전은 이러한 트렌드를 반영하고 있으며, 고객이 접하는 모든 디지털 접점을 최적의 경험으로 이끄는 데 기여하고 싶습니다. 전자공학 전공을 통해 하드웨어와 소프트웨어의 통합 가능성을 실감했고, 사용자 중심 개발의 중요성을 깨닫게 되었습니다. 가비아의 소프트웨어 엔지니어로서 최신 기술을 활용하여 최고의 사용자 경험을 제공하는 것이 제 꿈입니다.

**경험 및 역량: 다양한 프로젝트로 다져온 소프트웨어 개발 실력**  
저는 웹 개발에 집중하며 React, TypeScript를 활용한 여러 프로젝트를 진행했습니다. 특히 <스마트 홈 관리 시스템> 프로젝트에서는 사용자 피드백을 반영해 UI/UX를 개선하고, 이로 인해 사용자 만족도를 30% 증가시켰습니다. Next.js를 활용하여 서버 사이드 렌더링(SSR)을 구현해 페이지 로딩 속도를 40% 줄이고, 사용자 이탈률을 크게 낮춘 경험이 있습니다. 

또한, 대학 시절 동아리에서 진행한 프로젝트에서는 Atomic Design 패턴을 적용해 UI 컴포넌트를 설계하며 Storybook을 활용하여 독립적인 컴포넌트 개발 방법을 익혔습니다. 이러한 경험들은 재사용성과 유지보수성을 고려한 개발 능력을 기르는데 큰 도움이 되었습니다.

**강점 및 보완 사항: 협업 능력과 성과 중심의 접근**  
저의 강점은 사용자 피드백을 적극적으로 반영하여 개선하는 능력입니다. 협업 과정에서 열린 마음으로 다른 개발자의 의견을 경청하고 최적의 해결책을 도출하는 것을 소중히 여깁니다. 그러나 지나친 완벽주의가 프로젝트 일정에 영향을 미칠 수 있다는 점은 저의 약점입니다. 이를 해결하기 위해, ’고-로우’ 목표 설정 전략을 활용하고, 우선 실행할 기능과 개선할 사항을 분류